<a href="https://colab.research.google.com/github/wiiiiish/2022-machine_learning/blob/main/s_mysql_aod_pred_insert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls

drive  sample_data


In [ ]:
%cd drive/MyDrive/Colab\ Notebooks/2022-04-27\ 데이터

/content/drive/MyDrive/Colab Notebooks/2022-04-27 데이터


In [ ]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
     |████████████████████████████████| 43 kB 2.5 MB/s 


In [ ]:
import pandas as pd
import pymysql # mysql 연결
import pickle # 모델 불러올 때 사용
from datetime import date, datetime, timezone, timedelta # 시간 관련 라이브러리

In [ ]:
from pymysql.converters import convert_bit

# mysql 연결
# 코드 모두 정상적인데 오류가 생긴다면, 이 코드 재실행 할 것!
host = "project-db-stu.ddns.net"
port = 3307
username = "wish"
password = "1234"
db = 'wish'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

In [ ]:
# 모델 불러오기
with open('s_model.pickle', 'rb') as f:
    s_model = pickle.load(f)

In [ ]:
# 시간 맞추기
# hours=9 는 현재 한국시간
# 우리는 한 시간 후의 시간을 불러와야 하기 때문에 hours=10 으로 설정
K_KST = timezone(timedelta(hours=9))
H_KST = timezone(timedelta(hours=10))

In [ ]:
# DB 연결
from pymysql.converters import convert_bit
host = "project-db-stu.ddns.net"
port = 3307
username = "wish"
password = "1234"
db = 'wish'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

# select 현재 시간, 데이터 번호 (real_data의 마지막 데이터 불러오기)
sql_state1 = f"select r_seq, r_date from s_real_data order by r_date desc limit 1"
s_real = pd.read_sql_query(sql_state1, conn)
s_r_seq = s_real['r_seq']
s_r_date = s_real['r_date']

# 불러온 데이터의 시간 추출
s_now_time = f"{s_r_date[0]}"
s_time = s_now_time[11:].replace(":","") # : 빼기

# s_now_time 의 한 시간 후의 시간
print(f"s_time : {s_time}")
print(f"s_now_time : {s_now_time}")

s_now_time_slice = s_now_time[:18]
s_pred_t = (int)(s_now_time_slice[11:13]) + 1

if s_pred_t < 10:
  s_pred_t = f"0{s_pred_t}"

timeList = list(s_now_time)
timeList[11:13] = f"{s_pred_t}"
s_pred_time = ''.join(s for s in timeList)
s_pred_time_slice = s_pred_time[:18]
s_pred_time_slice

# select 한 시간 후의 날씨
sql_state2 = f"select * from temp_weather where date_time like '{s_pred_time_slice}%'"
s_after_weather = pd.read_sql_query(sql_state2, conn)

# 불러온 데이터의 시간 형식 변경
date_time = s_after_weather['date_time']
s_time = date_time[0][11:].replace(":","") # : 빼기
print(s_time) # 변경된 시간

# 변경된 시간 넣기
s_after_weather['date_time'] = s_time
print(s_after_weather)

# 발전량 예측 (모델에 대입)
s_pred_aod = s_model.predict(s_after_weather)
s_pred_aod[0] # 예측 발전량

# DB insert into s_predict
sql_state3 = f"insert into s_prediction values (null, {s_r_seq}, '{s_pred_time}', {s_pred_aod})"
cursor = conn.cursor()
cursor.execute(sql_state3)
conn.commit()

In [ ]:
# 1. 시간 구하기

# 현재 시간 구하기
now = datetime.now(K_KST)
current_time = now.strftime('%Y-%m-%d %H:%M:%S')
s_current_time = f"{current_time}"
s_current_time = s_current_time[:18]
print(s_current_time) # 현재 시간 / 11:11:12 -> 11:11:1 로 자르기

# 예측 시간 구하기
pred = datetime.now(H_KST)
pred_time = now.strftime('%Y-%m-%d %H:%M:%S')
s_pred_time = f"{pred_time}"
s_pred_time = s_pred_time[:18]

# 2. 1시간 후의 날씨+시간 데이터 불러오기
sql_state2 = f"select * from temp_weather where date_time like '{s_pred_time}%'"
s_after_weather = pd.read_sql_query(sql_state2, conn)
print(s_after_weather) # DB에서 불러온 데이터

# 불러온 시간 데이터 형식 변경
#(2022-05-04 11:00:00 형식) -> (110000) 으로 변경
date_time = s_after_weather['date_time']
# 시간 데이터 추출
s_time = date_time[0][11:].replace(":","") # : 빼기
print(s_time) # 변경된 시간

# 변경된 시간 넣기
s_after_weather['date_time'] = s_time
print(s_after_weather)